<a href="https://colab.research.google.com/github/SadafShafi/Transfer-learning-udacity-tensorflow/blob/main/transfer_learning_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers

In [4]:
(training_set, validation_set), dataset_info = tfds.load('tf_flowers',
                                                         split=['train[70%:]','train[:30%]'],
                                                         with_info=True,
                                                         as_supervised=True,
                                                         )

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


In [5]:
num_classes = 5
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4", output_shape=[2048],
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.build([None, 299, 299, 3])  # Batch input shape.

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 21,813,029
Trainable params: 10,245
Non-trainable params: 21,802,784
_________________________________________________________________


In [7]:
IMAGE_RES = 224

n = 0
for a in training_set:
  n+=1
num_training_examples = n

m = 0
for a in validation_set:
  m+=1

  
num_validation_examples = m
num_classes = dataset_info.features['label'].num_classes


def format_image(image, label):

  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label


BATCH_SIZE = 32
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [8]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
EPOCHS = 6

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/6
35/35 [==============================] - 14s 388ms/step - loss: 1.3926 - accuracy: 0.5223 - val_loss: 1.2305 - val_accuracy: 0.6966
Epoch 2/6
35/35 [==============================] - 12s 344ms/step - loss: 1.1594 - accuracy: 0.7702 - val_loss: 1.1520 - val_accuracy: 0.7784
Epoch 3/6
35/35 [==============================] - 12s 345ms/step - loss: 1.1005 - accuracy: 0.8338 - val_loss: 1.1273 - val_accuracy: 0.8047
Epoch 4/6
35/35 [==============================] - 12s 344ms/step - loss: 1.0647 - accuracy: 0.8719 - val_loss: 1.1091 - val_accuracy: 0.8220
Epoch 5/6
35/35 [==============================] - 12s 345ms/step - loss: 1.0420 - accuracy: 0.8965 - val_loss: 1.0980 - val_accuracy: 0.8292
Epoch 6/6
35/35 [==============================] - 12s 345ms/step - loss: 1.0212 - accuracy: 0.9173 - val_loss: 1.0947 - val_accuracy: 0.8229


In [10]:
image_batch, label_batch = next(iter(train_batches))
class_names = np.array(dataset_info.features['label'].names)
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()

predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

In [11]:
print("Labels:           ", label_batch)
print("Predicted labels: ", predicted_ids)

Labels:            [0 1 1 2 0 0 1 2 3 1 0 4 3 0 0 4 3 4 2 3 1 0 2 3 1 1 3 1 3 2 1 1]
Predicted labels:  [0 1 1 2 0 0 1 2 3 1 0 4 3 0 0 4 3 4 2 3 1 0 2 3 1 1 3 1 3 2 1 1]
